In [2]:
import pandas as pd
import numpy as np
from sklearn import metrics
from data_analysis import get_start, get_start_end
import math

In [3]:
filepath = "../Data/Combined/FHD_55.csv"
data = pd.read_csv(filepath, index_col=False)

In [4]:
#for val in data:
#    if val != 'time':
#        data[val] = (data[val] - min(data[val])) / (max(data[val]) - min(data[val]))

In [5]:
data['controller_right_vel'] = (data['controller_right_vel.x'] ** 2 + data['controller_right_vel.y'] ** 2 + data['controller_right_vel.z'] ** 2) ** (1/2)

In [6]:
idx_max = -1
mx = -float('inf')
for i in range(len(data['controller_right_vel'])):
    if float(data['controller_right_vel'][i]) > mx:
        mx = float(data['controller_right_vel'][i])
        idx_max = i

In [7]:
mx, idx_max

(10.075528058846395, 103)

In [8]:
filepath = "../Data/Combined/SRV_50.csv"
data = pd.read_csv(filepath, index_col=False)

data['controller_right_vel'] = (data['controller_right_vel.x'] ** 2 + data['controller_right_vel.y'] ** 2 + data['controller_right_vel.z'] ** 2) ** (1/2)
data['controller_right_rel_headset'] = (data['headset_pos.z']) - (data['controller_right_pos.z'])


idx_max = -1
mx = -float('inf')
for i in range(len(data['controller_right_vel'])):
    if float(data['controller_right_vel'][i]) > mx:
        mx = float(data['controller_right_vel'][i])
        idx_max = i

print(data['controller_right_vel.y'][idx_max])
print(data['controller_right_vel.x'][idx_max])
print(data['controller_left_vel.z'][idx_max])

-11.74106
0.1610769
-2.119427


In [9]:
def simple_stat_classifier(filepath):
    data = pd.read_csv(filepath, index_col=False)
    data['controller_right_vel'] = (data['controller_right_vel.x'] ** 2 + data['controller_right_vel.y'] ** 2 + data['controller_right_vel.z'] ** 2) ** (1/2)

    idx_max = -1
    max = -float('inf')
    for i in range(len(data['controller_right_vel'])):
        if float(data['controller_right_vel'][i]) > max:
            max = float(data['controller_right_vel'][i])
            idx_max = i

    if float(data['controller_right_vel.y'][idx_max]) > 0:
        # BHD or FHD
        if float(data['controller_right_vel.x'][idx_max]) < 0:
            return "FHD"
        else:
            return "BHD"
    else:
        # SRV or VOL
        if float(data['controller_right_vel.y'][idx_max]) < -3.5: # Really wouldn't want to use this part...
            return "SRV"
        else:
            return "VOL"

In [10]:
simple_stat_classifier("../Data/Combined/BHD_13.csv")

'BHD'

Preliminary Data Analysis seems to show that if we just take the controller_right_vel.y datapoint at the highest controller velocity, we can differentiate between serves and FHD/BHD/VOL. 

SRV is highly negative. (controller moving downwards)

FHD, BHD, are highly positive. (controller moving upwards)

VOL is slightly negative. (controller moving downwards)

Then out of FHD, BHD, we can check controller_right_vel.x
 - If > 0, BHD. (Controller moving to the right)
 - If < 0, FHD. (Controller moving to the left)

But magnitude is dependent on person, so we might not be able to use that... Might only be able to use +/-

In [11]:
def test_accuracy(classifier, errors=False):
    """
    Function to test how accurate your classifier is. Takes a classifier 
    (a function) that takes a filepath, and returns a string, either 
    'BHD', 'FHD', 'VOL', or 'SRV'. 

    Also can specifiy whether to print errors.

    Returns accuracy score of the classifier. 
    """
    actual = []
    predicted = []
    csv_number = []
    for action in ['BHD','FHD','VOL','SRV']:
        for i in range(1,91):
            filepath = "../Data/Combined/" + action + "_" + str(i).zfill(2) + ".csv"
            actual.append(action)
            predicted.append(classifier(filepath))
            csv_number.append(i)
    
    if errors:
        for i in range(len(actual)):
            if actual[i] != predicted[i]:
                print(actual[i], predicted[i], csv_number[i])
    
    return metrics.accuracy_score(actual, predicted)

In [12]:
def simple_stat_classifier_2(filepath):
    """
    Takes a filepath containing a sensor trace. First trims data according to
    isolate the swing action. Then, based on trimmed csv file, runs statistical
    classifier to determine swing type.

    Returns string, determining guessed swing type.
    """
    data = pd.read_csv(filepath, index_col=False)
    data['controller_right_vel'] = (data['controller_right_vel.x'] ** 2 + data['controller_right_vel.y'] ** 2 + data['controller_right_vel.z'] ** 2) ** (1/2)

    start, end = get_start_end(data) # Determine start and end of swing
    data = data[start:end]
    
    idx_max = -1
    mx = -float('inf') # Finding moment of maximum velocity
    for i in range(len(data['controller_right_vel'])):
        i += start
        if float(data['controller_right_vel'][i]) > mx:
            mx = float(data['controller_right_vel'][i])
            idx_max = i

    if float(data['controller_right_vel.y'][idx_max]) > 0: # If swing moves upwards
        # BHD or FHD
        if float(data['controller_right_vel.x'][idx_max]) < 0: # If swing moves left/right
            return "FHD"
        else:
            return "BHD"
    else:
        # SRV or VOL    # Finding peak of controller, relative to headset
        if max((data['controller_right_pos.y'] - data['headset_pos.y'])) > 0.2: # 0.2 may need to be normalized
        #if max((data['controller_right_pos.y'] - data['headset_pos.y'])) > 0: # This works for Ian/Lucien, but gets Jason wrong every single time lol
            return "SRV"
        else:
            return "VOL"

TESTING BELOW

In [13]:
filepath = "../Data/Combined/SRV_57.csv"
data = pd.read_csv(filepath, index_col=False)

data['controller_right_vel'] = (data['controller_right_vel.x'] ** 2 + data['controller_right_vel.y'] ** 2 + data['controller_right_vel.z'] ** 2) ** (1/2)

idx_max = -1
mx = -float('inf')
for i in range(len(data['controller_right_vel'])):
    if float(data['controller_right_vel'][i]) > mx:
        mx = float(data['controller_right_vel'][i])
        idx_max = i

In [14]:
s = get_start(data)
data['headset_pos.y'][s] - data['controller_right_pos.y'][s]

-0.40462167

In [15]:
data['headset_pos.y'].mean()

0.015195051647058843

In [16]:
test_accuracy(simple_stat_classifier)

0.9805555555555555

In [17]:
test_accuracy(simple_stat_classifier_2)

0.9944444444444445

In [18]:
def get_start_end_after_classification(data, swing):
    """
    Another preliminary Function to determine start and end indices for a given FHD
    or BHD sensor trace.

    Returns tuple, of indices at which start and end of swing are estimated
    to be.
    """
    start = 0
    end = len(data["controller_right_vel.z"])

    data["controller_right_vel"] = (
        data["controller_right_vel.x"] ** 2
        + data["controller_right_vel.y"] ** 2
        + data["controller_right_vel.z"] ** 2
    ) ** (1 / 2)

    pos = -1
    neg = -1
    mn = float("inf")
    mx = -float("inf")
    apex = -1
    max_apex = -float("inf")

    for i in range(
        len(data["controller_right_vel"])
    ):  # Finding moment of highest Velocity
        if float(data["controller_right_vel"][i]) > max_apex:
            apex = i
            max_apex = data["controller_right_vel"][i]

    for i in range(apex):
        if (
            float(data["controller_right_vel.z"][i]) > mx
        ):  # Finding moment of highest velocity forwards, up to apex
            mx = float(data["controller_right_vel.z"][i])
            pos = i
    for i in range(
        apex, len(data["controller_right_vel.z"])
    ):  # Finding moment of highest velocity backwards, after apex
        if float(data["controller_right_vel.z"][i]) < mn:
            mn = float(data["controller_right_vel.z"][i])
            neg = i

    for i in range(pos, 0, -1):  
        # Search backwards from moment of highest velocity forwards. Finds moment when z-velocity becomes significant.
        if (
            data["controller_right_vel.z"][i] >= 0.2
            and data["controller_right_vel.z"][i - 1] < 0.2
        ):
            start = i
            break

    if (
        swing != 'VOL'
    ):  
        for i in range(
            neg, len(data["controller_right_vel.z"]) - 1
        ):  # Search forwards from moment of highest velocity backwards. Finds moment when z-velocity becomes insignificant.
            if (
                data["controller_right_vel.z"][i] <= -0.2
                and data["controller_right_vel.z"][i + 1] > -0.2
            ):
                end = i + 1
                break

    else:
        for i in range(
            pos, len(data["controller_right_vel.z"]) - 1
        ):  # Search forwards from moment of highest velocity forwards. Finds moment when z-velocity becomes insignificant.
            if (
                data["controller_right_vel.z"][i] >= 0.2
                and data["controller_right_vel.z"][i + 1] < 0.2
            ):
                end = i + 1
                break
    
    end = min(end, len(data) - 1)

    return (start, end)

In [19]:
filepath = "../Data/Combined/FHD_25.csv"
data = pd.read_csv(filepath, index_col=False)

prediction = simple_stat_classifier_2(filepath)

print(prediction)

start, end = get_start_end_after_classification(data, prediction)
print(start, end)

FHD
96 142


In [20]:
def get_rotation(filepath, prediction=None, start_end=None):
    """
    Script to get controller rotation around the body, given a filepath to a 
    CSV sensor trace.

    Takes optional parameters:
        prediction: (str): ("SRV", "FHD", "BHD", "VOL")
        start_end: (tuple(int, int)): start and end of swing
    
    If these optional parameters are not given, they are computed and then used.

    Returns the degrees of rotation that the controller goes through.
    """
    data = pd.read_csv(filepath, index_col=False)
    if not prediction:
        prediction = simple_stat_classifier_2(filepath)
    if not start_end:
        start, end = get_start_end_after_classification(data, prediction)
    else:
        start, end = start_end

    res = 0
    for i in range(start, end-2):
        vec1 = (data['controller_right_pos.x'][i] - data['headset_pos.x'][i], data['controller_right_pos.z'][i] - data['headset_pos.z'][i])
        vec2 = (data['controller_right_pos.x'][i+1] - data['headset_pos.x'][i+1], data['controller_right_pos.z'][i+1] - data['headset_pos.z'][i+1])

        dot = vec1[0] * vec2[0] + vec1[1] * vec2[1]
        mag1 = (vec1[0] ** 2 + vec1[1] ** 2) ** 0.5
        mag2 = (vec2[0] ** 2 + vec2[1] ** 2) ** 0.5
        res += math.acos(dot/mag1/mag2)

    return res * 180 / math.pi

In [21]:
print(data['controller_right_vel.z'].to_string())

0     -0.001610
1     -0.006126
2     -0.012179
3     -0.019597
4     -0.028795
5     -0.040843
6     -0.058095
7     -0.066473
8     -0.066762
9     -0.062341
10    -0.059490
11    -0.063195
12    -0.066854
13    -0.071403
14    -0.083521
15    -0.098973
16    -0.112727
17    -0.127699
18    -0.149924
19    -0.167141
20    -0.180358
21    -0.181333
22    -0.182009
23    -0.191836
24    -0.207189
25    -0.227402
26    -0.244742
27    -0.263510
28    -0.284230
29    -0.303490
30    -0.321675
31    -0.336622
32    -0.354128
33    -0.383165
34    -0.418645
35    -0.451937
36    -0.480602
37    -0.511104
38    -0.546472
39    -0.582922
40    -0.622702
41    -0.675475
42    -0.731499
43    -0.793885
44    -0.866726
45    -0.947489
46    -1.029217
47    -1.115293
48    -1.193884
49    -1.289700
50    -1.368237
51    -1.446350
52    -1.529793
53    -1.615017
54    -1.691071
55    -1.752964
56    -1.813224
57    -1.874460
58    -1.934296
59    -1.983339
60    -2.007944
61    -2.006197
62    -1

In [22]:
x = -cos(yaw)sin(pitch)sin(roll)-sin(yaw)cos(roll) + 
y = -sin(yaw)sin(pitch)sin(roll)+cos(yaw)cos(roll)
z =  cos(pitch)sin(roll)

SyntaxError: invalid syntax (3165665707.py, line 1)

In [ ]:
import numpy as np

In [ ]:
for point in range(start, end):

    y = data['controller_right_rot.x'][point]
    p = data['controller_right_rot.y'][point]
    r = data['controller_right_rot.z'][point]

    vx = - np.cos(y) * np.sin(p) * np.sin(r) - np.sin(y) * np.cos(r) 
    vy =  - np.sin(y) * np.sin(p) * np.sin(r) + np.cos(y) * np.cos(r)
    vz = np.cos(p) * np.sin(r)

    print(vx, vy, vz)

0.4436927611541268 -0.7837673476204984 -0.4345635494416851
0.4188479765295091 -0.8290369610646697 0.3704916864732503
0.5737174630565332 0.805094989519917 -0.1505666976256712
0.9330389232626198 0.1581655472941278 -0.3231439730616682
0.1861495700070767 -0.2820960996415709 -0.9411536155980029
-0.9739730591816456 -0.22452553602085112 -0.031060644920802703
0.8791926623202106 0.16874426556131333 -0.4455845995571134
-0.26512627487282986 -0.36297767775733564 -0.893283417411265
-0.21149174823434386 0.9293434910095277 -0.3026415638127133
0.0027754246495246314 0.9998554159266531 0.016776300550317993
0.5232662996988514 -0.6866536744849975 -0.5046772343941451
0.299669289622962 -0.9540427151055045 -0.0009025580238156073
0.14365733595873217 0.16685470502757602 -0.9754599311275685
-0.5595778562843411 -0.8268225929146273 0.05689483809761267
0.9916613127014255 -0.07279670567692367 0.10634133972201434
-0.5845194037184838 -0.04354716862244004 0.8102102880003172
0.13833356917118506 0.6182814258920029 -0.77

In [ ]:
y = data['controller_right_rot.x'][end]
p = data['controller_right_rot.y'][end]
r = data['controller_right_rot.z'][end]

vx = - np.cos(y) * np.sin(p) * np.sin(r) - np.sin(y) * np.cos(r) 
vy =  - np.sin(y) * np.sin(p) * np.sin(r) + np.cos(y) * np.cos(r)
vz = np.cos(p) * np.sin(r)

(vx, vy, vz)

(-0.7353439134026587, -0.3406624545073839, -0.5858484625828317)

In [ ]:
yaw = data['controller_right_rot.x']
pitch = data['controller_right_rot.y']
roll = data['controller_right_rot.z']

In [ ]:
data['racquet_head.x'] = data['controller_right_pos.x']  - np.cos(yaw) * np.sin(pitch) * np.sin(roll) - np.sin(yaw) * np.cos(roll) 
data['racquet_head.y'] = data['controller_right_pos.y'] - np.sin(yaw) * np.sin(pitch) * np.sin(roll) + np.cos(yaw) * np.cos(roll)
data['racquet_head.z'] = data['racquet_head.z'] + np.cos(pitch) * np.sin(roll)

In [ ]:
print(data[['controller_right_pos.x', 'racquet_head.x']].to_string())

     controller_right_pos.x  racquet_head.x
0                 -0.105218       -0.428615
1                 -0.110815       -0.798629
2                 -0.115712       -1.035955
3                 -0.120331       -1.108815
4                 -0.124608       -1.090435
5                 -0.128247       -0.899571
6                 -0.131261       -0.092878
7                 -0.133856       -0.349552
8                 -0.135337       -1.131021
9                 -0.136751       -0.015358
10                -0.137006        0.779348
11                -0.136466       -0.653163
12                -0.135227       -0.666784
13                -0.133517        0.677437
14                -0.130830        0.069562
15                -0.127354       -1.122987
16                -0.123440        0.363527
17                -0.118846        0.111146
18                -0.113292       -0.796391
19                -0.107225        0.433817
20                -0.100573       -0.769706
21                -0.093377     

In [ ]:
print(data[['controller_right_rot.x', 'controller_right_rot.y', 'controller_right_rot.z']].iloc[start:end].to_string())

     controller_right_rot.x  controller_right_rot.y  controller_right_rot.z
96               358.550700              226.410500               28.735380
97                 0.817710              226.904600               27.763950
98                 2.518017              226.223800               28.425540
99                 3.688553              224.976100               30.202890
100                4.220399              223.222200               32.684830
101                5.039543              221.782400               34.452260
102                7.348401              220.821700               35.370310
103               11.218020              220.231000               35.782810
104               16.453660              218.907800               35.155980
105               21.982130              216.413700               34.575420
106               26.425900              212.830900               35.364850
107               28.597310              208.867500               37.680460
108         

In [ ]:
print(start, end)

96 142


In [ ]:
print(data['controller_right_rot.y'][start:end].to_string())

96     226.410500
97     226.904600
98     226.223800
99     224.976100
100    223.222200
101    221.782400
102    220.821700
103    220.231000
104    218.907800
105    216.413700
106    212.830900
107    208.867500
108    204.122600
109    197.791500
110    189.622200
111    179.376400
112    166.322600
113    152.214600
114    135.420300
115    115.277000
116     94.753040
117     76.093280
118     60.585140
119     47.445190
120     36.419990
121     26.398250
122     16.026680
123      5.420216
124    354.471400
125    342.616700
126    329.766800
127    317.529800
128    304.018600
129    285.760700
130    266.065200
131    248.919300
132    236.431100
133    226.751700
134    217.952400
135    209.634100
136    202.355900
137    195.529300
138    190.262600
139    186.867200
140    184.246800
141    182.281800


In [ ]:
print((data['headset_pos.x'][start], data['headset_pos.z'][start]))
print((data['headset_pos.x'][end], data['headset_pos.z'][end]))

(-0.4157858, 0.4973601)
(-0.3527169, 0.3221071)


In [ ]:
print((data['controller_right_pos.x'][start], data['controller_right_pos.z'][start]))
print((data['controller_right_pos.x'][end], data['controller_right_pos.z'][end]))

(-0.4463193, -0.03785687)
(-0.1404012, 0.1554287)


In [23]:
import pandas as pd

In [58]:
filepath = "../Data/Combined/FHD_52.csv"
data = pd.read_csv(filepath, index_col=False)
prediction = simple_stat_classifier_2(filepath)
start, end = get_start_end_after_classification(data, prediction)

res = 0
for i in range(start, end-2):
    #vec1 = (data['controller_right_pos.x'][i] - data['headset_pos.x'][i], data['controller_right_pos.z'][i] - data['headset_pos.z'][i], 0)
    #vec2 = (data['controller_right_pos.x'][i+1] - data['headset_pos.x'][i+1], data['controller_right_pos.z'][i+1] - data['headset_pos.z'][i+1], 0)

    vec1 = (data['controller_right_pos.x'][i] - data['headset_pos.x'][start], data['controller_right_pos.z'][i] - data['headset_pos.z'][start], 0)
    vec2 = (data['controller_right_pos.x'][i+1] - data['headset_pos.x'][start], data['controller_right_pos.z'][i+1] - data['headset_pos.z'][start], 0)

    dot = vec1[0] * vec2[0] + vec1[1] * vec2[1]
    mag1 = (vec1[0] ** 2 + vec1[1] ** 2) ** 0.5
    mag2 = (vec2[0] ** 2 + vec2[1] ** 2) ** 0.5

    if prediction == "FHD":
        if np.cross(vec1, vec2)[2] > 0:
            res += math.acos(dot/mag1/mag2)
    elif prediction == "BHD":
        if np.cross(vec1, vec2)[2] < 0:
            res += math.acos(dot/mag1/mag2)
    else:
        res += math.acos(dot/mag1/mag2)

    # X.Y = |x||y|cos(angle)
    # angle = arccos[(X.Y)/(|x||y|)]
    #print(np.cross(vec1, vec2))

print(res * 180 / math.pi)

325.46474263753055


In [ ]:
print((data['headset_pos.x'][end], data['headset_pos.z'][end]))
print((data['controller_right_pos.x'][end], data['controller_right_pos.z'][end]))

(-0.03261396, -0.7293273)
(0.02082731, -0.3959379)


In [ ]:
filepath = "../Data/Combined/SRV_15.csv"
data = pd.read_csv(filepath, index_col=False)

prediction = simple_stat_classifier_2(filepath)
start, end = get_start_end_after_classification(data, prediction)

print((data['controller_right_pos.x'][end] - data['headset_pos.x'][end], data['controller_right_pos.z'][end] - data['headset_pos.z'][end], data['controller_right_pos.y'][end] - data['headset_pos.y'][end]))

(0.23239130000000002, -0.09658070000000007, -0.6436957)


In [ ]:
data['headset_pos.y'][end] - data['controller_right_pos.y'][end]


0.54600852

In [ ]:
max((data["controller_right_pos.y"] - data["headset_pos.y"]))

0.3229801

In [ ]:
def get_follow_through(filepath, prediction=None, start_end = None):
    data = pd.read_csv(filepath, index_col=False)

    if not prediction:
        prediction = simple_stat_classifier_2(filepath)
    if not start_end:
        _, end = get_start_end_after_classification(data, prediction)
    else:
        _, end = start_end

    tup = (data['controller_right_pos.x'][end] - data['headset_pos.x'][end], data['controller_right_pos.z'][end] - data['headset_pos.z'][end], data['controller_right_pos.y'][end] - data['headset_pos.y'][end])

    if prediction == "FHD":
        success = True
        if tup[2] < -0.2: # Checks height of controller vs. height of headset
            print("Try to follow-through a bit higher, over your shoulder!")
            success = False
        if tup[1] > 0: # Checks "forward depth" of controller vs. headset
            print("Try to end your swing further back!")
            success = False
        if tup[0] > 0: # Checks left/right position of contrller vs. headset
            print("Make sure to complete your follow-through on the left side of your body!")
            success = False
        if success:
            print("Nice follow through!")

    if prediction == "BHD":
        success = True
        if tup[2] < -0.2: # Checks height of controller vs. height of headset
            print("Try to follow-through a bit higher, over your shoulder!")
            success = False
        if tup[1] > 0: # Checks "forward depth" of controller vs. headset
            print("Try to end your swing further back!")
            success = False
        if tup[0] < 0: # Checks left/right position of contrller vs. headset
            print("Make sure to complete your follow-through on the right side of your body!")
            success = False
        if success:
            print("Nice follow through!")

    if prediction == "SRV":
        success = True
        if -1 * tup[2] < max(data["controller_right_pos.y"] - data["headset_pos.y"]):
            # Checks height of controller vs headset, makes sure difference is
            # at least as much as difference during peak of serve
            print("Try to finish a bit lower, near your waist!")
            success = False
        if tup[1] > 0: # Checks "forward depth" of controller vs. headset
            print("Try to end your swing further back!")
            success = False
        if tup[0] < 0: # Checks left/right position of contrller vs. headset
            print("Make sure to complete your follow-through on the left side of your body!")
            success = False
        if success:
            print("Nice follow through!")
    
    if prediction == "VOL":
        success = True
        if tup[2] > 0: # Checks height of controller vs. height of headset
            print("Try to keep your hand below your head in a volley!!")
            success = False
        if tup[1] < 0: # Checks "forward depth" of controller vs. headset
            print("Try to end your volley in front of your body!")
            success = False
        #if tup[0] < 0: # Checks left/right position of contrller vs. headset
        #    print("Make sure to finish on the right side of your body!")
        #    success = False
        if success:
            print("Nice shot!")


In [ ]:
filepath = "../Data/Combined/VOL_05.csv"
data = pd.read_csv(filepath, index_col=False)
prediction = simple_stat_classifier_2(filepath)
start, end = get_start_end_after_classification(data, prediction)

get_follow_through(filepath, prediction=prediction, start_end = (start, end))

Make sure to finish on the right side of your body!


In [ ]:
print((data['controller_right_pos.x'][end] - data['headset_pos.x'][end], data['controller_right_pos.z'][end] - data['headset_pos.z'][end], data['controller_right_pos.y'][end] - data['headset_pos.y'][end]))

(-0.20295658, 0.4413993999999999, -0.6343442)


In [ ]:
max(data["controller_right_pos.y"] - data["headset_pos.y"])

0.5284434400000001

In [ ]:
data['controller_right_rot.z'][end]

264.812

In [ ]:
filepath = "../Data/Combined/VOL_40.csv"
data = pd.read_csv(filepath, index_col=False)
prediction = simple_stat_classifier_2(filepath)
start, end = get_start_end_after_classification(data, prediction)

y = data['controller_right_rot.y'][end] * math.pi / 180
p = data['controller_right_rot.x'][end] * math.pi / 180
r = data['controller_right_rot.z'][end] * math.pi / 180

vx = - np.cos(y) * np.sin(p) * np.sin(r) - np.sin(y) * np.cos(r) 
vy =  - np.sin(y) * np.sin(p) * np.sin(r) + np.cos(y) * np.cos(r)
vz = np.cos(p) * np.sin(r)

print(vx, vy, vz)

-0.4263477190789411 0.3596916927833782 -0.8299695829238637
